<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> Simulación de procesos financieros

**Nombres:**
Mateo Verea Dorantes,
Alejandra Rico

**Fecha:** 22 de noviembre de 2021

**Expediente** : 
    if709396
    
**Profesor:** Oscar David Jaramillo Zuluaga
    
**Link Github**:https://github.com/Mateoverea/Tarea9-10_MVerea_ARico
    
# Tarea 9

Implementar el método de esquemas del trapecio, para valuar la opción call y put asiática con precio inicial, $S_0 = 100$, precio de ejercicio $K = 100$, tasa libre de riesgo $r = 0.10$, volatilidad $\sigma = 0.20$ y $T = 1$ año. Cuyo precio es $\approx 7.04$. Realizar la simulación en base a la siguiente tabla:
![imagen.png](attachment:imagen.png)

Observe que en esta tabla se encuentran los intervalos de confianza de la aproximación obtenida y además el tiempo de simulación que tarda en encontrar la respuesta cada método. 
- Se debe entonces realizar una simulación para la misma cantidad de trayectorias y número de pasos y construir una Dataframe de pandas para reportar todos los resultados obtenidos.**(70 puntos)**
- Compare los resultados obtenidos con los resultados arrojados por la función `Riemann_approach`. Concluya. **(30 puntos)**

In [1]:
# Importamos librerías a utilizar.
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
import time
%matplotlib inline

# Algunas opciones para Pandas.
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 9)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 78)
pd.set_option('precision', 3)

#### Código de solución `ESTUDIANTE 1: MATEO VEREA`

In [2]:
# Datos escenario: 
np.random.seed(5555)
S0 = 100; K = 100 ; r = 0.10
sigma = 0.20; T = 1 ;p = 7.04
NbTraj = [1000, 5000, 10000, 50000, 100000, 500000, 1000000]
NbStep = [10, 50, 100]

In [7]:
def BSprices(mu,sigma,S0,NbTraj,NbStep):
    """
    Expresión de la solución de la ecuación de Black-Scholes
    St = S0*exp((r-sigma^2/2)*t+ sigma*DeltaW)
    
    Parámetros
    ---------
    mu    : Tasa libre de riesgo
    sigma : Desviación estándar de los rendimientos
    S0    : Precio inicial del activo subyacente
    NbTraj: Cantidad de trayectorias a simular
    NbStep: Número de días a simular
    """
    # Datos para la fórmula de St
    nu = mu-(sigma**2)/2
    DeltaT = 1/NbStep
    SqDeltaT = np.sqrt(DeltaT)
    DeltaW = SqDeltaT*np.random.randn(NbTraj,NbStep-1)
    
    # Se obtiene --> Ln St = Ln S0+ nu*DeltaT + sigma*DeltaW
    increments = nu*DeltaT + sigma*DeltaW
    concat = np.concatenate((np.log(S0)*np.ones([NbTraj,1]),increments),axis=1)
    
    # Se utiliza cumsum por que se quiere simular los precios iniciando desde S0
    LogSt = np.cumsum(concat,axis=1)
    # Se obtienen los precios simulados para los NbStep fijados
    St = np.exp(LogSt)
    # Vector con la cantidad de días simulados
    t = np.arange(0,NbStep)

    return St.T,t

def calc_daily_ret(closes):
    return np.log(closes/closes.shift(1)).iloc[1:]

In [8]:
def trapecio_p(K,r,S0,NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',sigma:'Volatilidad',T:'Tiempo de cierre del contrato en años',tipo:"put o call"=None):
    start = time.time()
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    prices = pd.DataFrame(St,index=t)
    strike = K
    h = T/NbStep
    formula = (2+r*h+np.random.randn(NbStep,NbTraj)*sigma)
    av1 = np.cumsum(prices*(formula))
    av2 = h/(2*T)
    Average_t = av1*av2
    strike = K
    #si es put:
    if tipo == "put":
        put_opt = pd.DataFrame({"Prima":np.exp(-r*T)*np.fmax(strike-Average_t,0).mean(axis=1)},index=t)
    else:
        put_opt = pd.DataFrame({"Prima":np.exp(-r*T)*np.fmax(Average_t-strike,0).mean(axis=1)},index=t)
    # intervalos de confianza
    confianza = 0.95
    sigma_est = put_opt.sem().Prima
    mean_est = put_opt.iloc[-1].Prima
    i1 = st.norm.interval(confianza,loc=mean_est,scale=sigma_est)
    distance = (i1[1]-i1[0])
    distance2 = np.round(distance,4)
    end = time.time()
    time_dif = int(end-start)
    timef = '{:02d}:{:02d}'.format((time_dif%3600//60),time_dif%60)
    total_time = lambda x: '< 1 s'if x=='00:00' else x #también es para el formato
    il0=np.round(i1[0],4)
    il1=np.round(i1[1],4)
    res = np.array([np.round(put_opt.iloc[-1].Prima,4),il0,il1,distance2,total_time(timef)])
    return res
#Mapeo put trapecio: 
NbTraj = [1000, 5000, 10000, 50000, 100000, 500000, 1000000]
NbStep = [10, 50, 100]
trap_p =  np.array(list(map(lambda N_traj:list(map(lambda N_step:trapecio_p(K,r,S0,N_traj,N_step,sigma,T,"put"),NbStep)),NbTraj)))
num = (0, 1, 2)
pd.set_option("display.max_rows", None, "display.max_columns", None)
ciclo =[i for i in NbTraj for _ in num]
rs = np.array(list(map(lambda i: trap_p[:,:,i].flatten(),range(5))))
rs0 = np.round(rs[0].astype('float'),4)
rs1 = np.round(rs[1].astype('float'),4)
rs2 = np.round(rs[2].astype('float'),4)
rs3 = np.round(rs[3].astype('float'),4)
listas = [ciclo,(len(NbTraj)*NbStep),rs0,rs1,rs2,rs3,rs[4]]
index = pd.MultiIndex.from_arrays(listas,names=('Tray.Monte Carlo','Núm.Pasos en el tiempo','Aproximación','Linferior','Lsuperior','Longitud al 95%','Tiempo (mm::ss)'))
df_trapecio_p = pd.DataFrame(index=index)

def trapecio_c(K,r,S0,NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',sigma:'Volatilidad',T:'Tiempo de cierre del contrato en años',tipo:"put o call"=None):
    start = time.time()
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    prices = pd.DataFrame(St,index=t)
    strike = K
    h = (T/NbStep)
    formula = (2+r*h+np.random.randn(NbStep,NbTraj)*sigma)
    Av1 = np.cumsum(prices*(formula))
    Av2 = h/(2*T)
    Average_t = Av1*Av2
    strike = K
    #si es call:
    if tipo == "call":
        call_opt = pd.DataFrame({"Prima": np.exp(-r*T)*np.fmax(Average_t-strike,0).mean(axis=1)},index=t)
    else:
        call_opt = pd.DataFrame({"Prima": np.exp(-r*T)*np.fmax(strike-Average_t,0).mean(axis=1)},index=t)
    # intervalos de confianza
    confianza = 0.95
    sigma_est = call_opt.sem().Prima
    mean_est = call_opt.iloc[-1].Prima
    i1 = st.norm.interval(confianza,loc=mean_est,scale=sigma_est)
    distance = (i1[1]-i1[0])
    distance2 = np.round(distance,4)
    end = time.time()
    time_dif = int(end - start)
    timef = '{:02d}:{:02d}'.format((time_dif%3600//60),time_dif%60)
    total_time = (lambda x: '< 1 s' if x=='00:00' else x)
    il0 = np.round(i1[0],4)
    il1 = np.round(i1[1],4)
    res = np.array([np.round(call_opt.iloc[-1].Prima,4),il0,il1,distance2,total_time(timef)])
    return res

NbTraj = [1000, 5000, 10000, 50000, 100000, 500000, 1000000]
NbStep = [10, 50, 100]
trap_c=np.array(list(map(lambda N_traj:list(map(lambda N_step: trapecio_c(K,r,S0,N_traj,N_step,sigma,T,"call"),NbStep)),NbTraj)))
num = (0, 1, 2)
pd.set_option("display.max_rows", None, "display.max_columns", None)
ciclo =[i for i in NbTraj for _ in num]
rs = np.array(list(map(lambda i: trap_c[:,:,i].flatten(),range(5))))
rs0 = np.round(rs[0].astype('float'),4)
rs1 = np.round(rs[1].astype('float'),4)
rs2 = np.round(rs[2].astype('float'),4)
rs3 = np.round(rs[3].astype('float'),4)
listas = [ciclo,(len(NbTraj)*NbStep),rs0,rs1,rs2,rs3,rs[4]]
index = pd.MultiIndex.from_arrays(listas, names=('Tray.Monte Carlo','Núm.Pasos en el tiempo','Aproximación','Linferior','Lsuperior','Longitud al 95%','Tiempo (mm::ss)'))
df_trapecio_c = pd.DataFrame(index=index)

#Función de clase adaptada para put:
def riemann_p(K,r,S0,NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',sigma:'Volatilidad',T:'Tiempo de cierre del contrato en años',tipo:"put o call"=None):
    start = time.time()
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    prices = pd.DataFrame(St,index=t)
    Average_t = prices.expanding().mean()
    strike = K
    #si es put:
    if tipo == "put":
        put_opt = pd.DataFrame({"Prima": np.exp(-r*T)*np.fmax(strike-Average_t,0).mean(axis=1)},index=t)
    else:
        put_opt = pd.DataFrame({"Prima": np.exp(-r*T)*np.fmax(Average_t-strike,0).mean(axis=1)},index=t)
    # intervalos de confianza
    confianza = 0.95
    sigma_est = put_opt.sem().Prima
    mean_est = put_opt.iloc[-1].Prima
    i1 = st.norm.interval(confianza,loc=mean_est,scale=sigma_est)
    il0 = np.round(i1[0],4)
    il1 = np.round(i1[1],4)
    distance = np.round(i1[1] - i1[0],4)
    distance2 = np.round(distance,4)
    end = time.time()
    time_dif = int(end - start)
    timef = '{:02d}:{:02d}'.format((time_dif%3600//60),time_dif%60)
    total_time = lambda x: '< 1 s' if x=='00:00' else x
    res = np.array([np.round(put_opt.iloc[-1].Prima,4),il0,il1,distance2,total_time(timef)])
    return res
# Mapeo Riemann para put:
NbTraj = [1000, 5000, 10000, 50000, 100000, 500000, 1000000]
NbStep = [10, 50, 100]
ri_p =  np.array(list(map(lambda N_traj:list(map(lambda N_step:riemann_p(K,r,S0,N_traj,N_step,sigma,T,'put'),NbStep)),NbTraj)))
num = (0, 1, 2)
pd.set_option("display.max_rows", None, "display.max_columns", None)
ciclo =[i for i in NbTraj for _ in num]
rs = np.array(list(map(lambda i: ri_p[:,:,i].flatten(),range(5))))
rs0 = np.round(rs[0].astype('float'),4)
rs1 = np.round(rs[1].astype('float'),4)
rs2 = np.round(rs[2].astype('float'),4)
rs3 = np.round(rs[3].astype('float'),4)
listas = [ciclo,(len(NbTraj)*NbStep),rs0,rs1,rs2,rs3,rs[4]]
index = pd.MultiIndex.from_arrays(listas, names=('Tray.Monte Carlo','Núm.Pasos en el tiempo','Aproximación','Linferior','Lsuperior','Longitud al 95%','Tiempo (mm::ss)'))
df_riemann_p = pd.DataFrame(index=index)

#Función de clase adaptada para call:
def riemann_c(K,r,S0,NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',sigma:'Volatilidad',T:'Tiempo de cierre del contrato en años',tipo:"put o call"=None):
    start = time.time()
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    prices = pd.DataFrame(St,index=t)
    Average_t = prices.expanding().mean()
    strike = K
    #si es call:
    if tipo == "call":
        call_opt = pd.DataFrame({"Prima": np.exp(-r*T)*np.fmax(Average_t - strike,0).mean(axis=1)}, index=t)
    else:
        call_opt = pd.DataFrame({"Prima": np.exp(-r*T)*np.fmax(Average_t - strike,0).mean(axis=1)}, index=t)
    # intervalos de confianza
    confianza = 0.95
    sigma_est = call_opt.sem().Prima
    mean_est = call_opt.iloc[-1].Prima
    i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
    il0 = np.round(i1[0],4)
    il1 = np.round(i1[1],4)
    distance = (i1[1]-i1[0])
    distance2 = np.round(distance,4)
    end = time.time()
    time_dif = int(end-start)
    timef = '{:02d}:{:02d}'.format((time_dif%3600//60),time_dif%60)
    total_time = lambda x: '<1 s' if x=='00:00' else x
    res = np.array([np.round(call_opt.iloc[-1].Prima,4),il0,il1,distance2,total_time(timef)])
    return res
# Mapeo Riemann para call:
NbTraj = [1000, 5000, 10000, 50000, 100000, 500000, 1000000]
NbStep = [10, 50, 100]
ri_c =  np.array(list(map(lambda N_traj:list(map(lambda N_step:riemann_c(K,r,S0,N_traj,N_step,sigma,T,'call'),NbStep)),NbTraj)))
num = (0, 1, 2)
pd.set_option("display.max_rows", None, "display.max_columns", None)
ciclo =[i for i in NbTraj for _ in num]
rs = np.array(list(map(lambda i: ri_c[:,:,i].flatten(),range(5))))
rs0 = np.round(rs[0].astype('float'),4)
rs1 = np.round(rs[1].astype('float'),4)
rs2 = np.round(rs[2].astype('float'),4)
rs3 = np.round(rs[3].astype('float'),4)
listas = [ciclo,(len(NbTraj)*NbStep),rs0,rs1,rs2,rs3,rs[4]]
index = pd.MultiIndex.from_arrays(listas, names=('Tray.Monte Carlo','Núm.Pasos en el tiempo','Aproximación','Linferior','Lsuperior','Longitud al 95%','Tiempo (mm::ss)'))
df_riemann_c = pd.DataFrame(index=index)

### Comparar put

In [9]:
df_trapecio_p 

Empty DataFrame
Columns: []
Index: [(1000, 10, 2.1528, -15.0221, 19.3276, 34.3497, < 1 s), (1000, 50, 2.2935, -5.1813, 9.7683, 14.9496, < 1 s), (1000, 100, 2.0869, -3.1885, 7.3622, 10.5507, < 1 s), (5000, 10, 2.2662, -14.8483, 19.3808, 34.2291, < 1 s), (5000, 50, 2.3017, -5.164, 9.7673, 14.9313, < 1 s), (5000, 100, 2.2905, -2.9716, 7.5526, 10.5242, < 1 s), (10000, 10, 2.2191, -14.9067, 19.345, 34.2517, < 1 s), (10000, 50, 2.3325, -5.1324, 9.7975, 14.9299, < 1 s), (10000, 100, 2.4046, -2.848, 7.6573, 10.5054, < 1 s), (50000, 10, 2.2194, -14.8984, 19.3372, 34.2356, < 1 s), (50000, 50, 2.3215, -5.144, 9.7871, 14.9311, < 1 s), (50000, 100, 2.3302, -2.9309, 7.5914, 10.5223, < 1 s), (100000, 10, 2.2092, -14.9118, 19.3302, 34.242, 00:01), (100000, 50, 2.3483, -5.1101, 9.8068, 14.9169, 00:01), (100000, 100, 2.3549, -2.9021, 7.6119, 10.514, 00:02), (500000, 10, 2.2272, -14.8894, 19.3438, 34.2331, 00:01), (500000, 50, 2.3334, -5.1282, 9.7949, 14.9231, 00:04), (500000, 100, 2.3397, -2.9181, 7.5975, 10.5156, 00:12), (1000000, 10, 2.2197, -14.8982, 19.3375, 34.2357, 00:04), (1000000, 50, 2.3315, -5.1305, 9.7936, 14.9242, 00:10), (1000000, 100, 2.3457, -2.9119, 7.6033, 10.5152, 00:49)]

### Con metodo de Riemann

In [10]:
df_riemann_p

Empty DataFrame
Columns: []
Index: [(1000, 10, 2.3023, 1.8608, 2.7438, 0.883, < 1 s), (1000, 50, 2.2623, 2.1184, 2.4062, 0.2878, < 1 s), (1000, 100, 2.3827, 2.2763, 2.4891, 0.2128, < 1 s), (5000, 10, 2.2026, 1.7813, 2.6239, 0.8426, < 1 s), (5000, 50, 2.2431, 2.1047, 2.3814, 0.2767, < 1 s), (5000, 100, 2.2555, 2.1624, 2.3487, 0.1863, < 1 s), (10000, 10, 2.1934, 1.7738, 2.6131, 0.8393, < 1 s), (10000, 50, 2.3578, 2.2075, 2.5082, 0.3006, < 1 s), (10000, 100, 2.3826, 2.2803, 2.485, 0.2048, < 1 s), (50000, 10, 2.2387, 1.8095, 2.6678, 0.8583, 00:01), (50000, 50, 2.3611, 2.2108, 2.5114, 0.3006, 00:01), (50000, 100, 2.3406, 2.2414, 2.4398, 0.1984, 00:01), (100000, 10, 2.247, 1.8163, 2.6776, 0.8612, 00:02), (100000, 50, 2.3454, 2.1967, 2.494, 0.2974, 00:02), (100000, 100, 2.3466, 2.2469, 2.4464, 0.1995, 00:03), (500000, 10, 2.2207, 1.7952, 2.6461, 0.851, 00:11), (500000, 50, 2.3335, 2.1855, 2.4816, 0.2961, 00:13), (500000, 100, 2.342, 2.2425, 2.4415, 0.199, 00:16), (1000000, 10, 2.2187, 1.7937, 2.6437, 0.8499, 00:22), (1000000, 50, 2.3363, 2.188, 2.4847, 0.2967, 00:26), (1000000, 100, 2.3492, 2.2493, 2.449, 0.1997, 00:53)]

### Comparar call

In [11]:
df_trapecio_c

Empty DataFrame
Columns: []
Index: [(1000, 10, 7.2226, 5.8059, 8.6394, 2.8335, < 1 s), (1000, 50, 6.8599, 6.4551, 7.2646, 0.8094, < 1 s), (1000, 100, 6.6397, 6.3803, 6.8991, 0.5188, < 1 s), (5000, 10, 6.8431, 5.5012, 8.185, 2.6839, < 1 s), (5000, 50, 7.1389, 6.7175, 7.5603, 0.8428, < 1 s), (5000, 100, 6.9681, 6.6943, 7.242, 0.5477, < 1 s), (10000, 10, 6.9804, 5.6118, 8.3489, 2.7372, < 1 s), (10000, 50, 6.8483, 6.4435, 7.253, 0.8094, < 1 s), (10000, 100, 7.0071, 6.7311, 7.283, 0.5519, < 1 s), (50000, 10, 6.8745, 5.5267, 8.2223, 2.6957, < 1 s), (50000, 50, 7.025, 6.6102, 7.4398, 0.8296, < 1 s), (50000, 100, 6.9888, 6.7142, 7.2633, 0.5492, 00:01), (100000, 10, 6.9213, 5.5637, 8.2788, 2.7151, < 1 s), (100000, 50, 7.0182, 6.6035, 7.4329, 0.8293, < 1 s), (100000, 100, 7.0401, 6.7632, 7.3169, 0.5537, 00:01), (500000, 10, 6.9074, 5.5525, 8.2622, 2.7097, 00:02), (500000, 50, 7.024, 6.6095, 7.4386, 0.8292, 00:05), (500000, 100, 7.0045, 6.7291, 7.2798, 0.5508, 00:08), (1000000, 10, 6.8945, 5.5423, 8.2466, 2.7043, 00:04), (1000000, 50, 7.0167, 6.6024, 7.431, 0.8285, 00:10), (1000000, 100, 7.0353, 6.7586, 7.312, 0.5533, 00:51)]

### Con Riemann

In [12]:
df_riemann_c

Empty DataFrame
Columns: []
Index: [(1000, 10, 6.2793, 5.0076, 7.551, 2.5434, <1 s), (1000, 50, 6.772, 6.2738, 7.2702, 0.9964, <1 s), (1000, 100, 6.8093, 6.4603, 7.1583, 0.698, <1 s), (5000, 10, 6.3644, 5.107, 7.6218, 2.5148, <1 s), (5000, 50, 6.8115, 6.3049, 7.3182, 1.0133, <1 s), (5000, 100, 6.9593, 6.603, 7.3157, 0.7127, <1 s), (10000, 10, 6.4959, 5.2044, 7.7873, 2.5829, <1 s), (10000, 50, 6.9491, 6.4315, 7.4667, 1.0352, <1 s), (10000, 100, 7.0507, 6.688, 7.4133, 0.7253, <1 s), (50000, 10, 6.3851, 5.1197, 7.6505, 2.5308, 00:01), (50000, 50, 6.9077, 6.3911, 7.4243, 1.0331, 00:01), (50000, 100, 6.9283, 6.5714, 7.2852, 0.7138, 00:01), (100000, 10, 6.4206, 5.1478, 7.6933, 2.5455, 00:02), (100000, 50, 6.8904, 6.3762, 7.4046, 1.0284, 00:02), (100000, 100, 6.9843, 6.6244, 7.3443, 0.7199, 00:03), (500000, 10, 6.4222, 5.1489, 7.6955, 2.5465, 00:11), (500000, 50, 6.9287, 6.4113, 7.4461, 1.0347, 00:13), (500000, 100, 6.9624, 6.604, 7.3208, 0.7168, 00:16), (1000000, 10, 6.4219, 5.1488, 7.6951, 2.5462, 00:21), (1000000, 50, 6.9089, 6.3933, 7.4244, 1.0311, 00:28), (1000000, 100, 6.9825, 6.6227, 7.3424, 0.7197, 01:00)]

- Para ambos put y call, el metodo del trapecio es más precisio, incluso es aproximadamente el doble mas eficiente por el tiempo usado en correr. 

#### Código de solución `ESTUDIANTE 2: ALEJANDRA RICO`

In [13]:
# Función para simular precios Black-Scholes.
def BSprices(mu,sigma,S0,NbTraj,NbStep):
    """
    Expresión de la solución de la ecuación de Black-Scholes
    St = S0*exp((r-sigma^2/2)*t+ sigma*DeltaW)
    
    Parámetros
    ---------
    mu    : Tasa libre de riesgo
    sigma : Desviación estándar de los rendimientos
    S0    : Precio inicial del activo subyacente
    NbTraj: Cantidad de trayectorias a simular
    NbStep: Número de días a simular
    """
    # Datos para la fórmula de St
    nu = mu-(sigma**2)/2
    DeltaT = 1/NbStep
    SqDeltaT = np.sqrt(DeltaT)
    DeltaW = SqDeltaT*np.random.randn(NbTraj,NbStep-1)
    
    # Se obtiene --> Ln St = Ln S0+ nu*DeltaT + sigma*DeltaW
    increments = nu*DeltaT + sigma*DeltaW
    concat = np.concatenate((np.log(S0)*np.ones([NbTraj,1]),increments),axis=1)
    
    # Se utiliza cumsum por que se quiere simular los precios iniciando desde S0
    LogSt = np.cumsum(concat,axis=1)
    # Se obtienen los precios simulados para los NbStep fijados
    St = np.exp(LogSt)
    # Vector con la cantidad de días simulados
    t = np.arange(0,NbStep)

    return St.T,t

$$\hat V_0^{(2)}= {e^{-rT} \over M} \sum_{j=1}^{M} \Bigg({h\over 2T} \sum_{i=0}^{N-1} S_{t_i}(2+rh+(W_{t_{i+1}}-W_{t_i})\sigma)-K \Bigg)_+,$$

In [14]:
# Función Método del Trapecio.
def Trapecio(K,r,S0,NbTraj,NbStep,sigma,T,Option_Type,Trust_level,flag=None):
    """
    K:           Strike price
    r:           Tasa libre de riesgo
    S0:          Precio inicial
    NbTraj:      Número trayectorias
    NbStep:      Cantidad de pasos a simular
    sigma:       Volatilidad
    T:           Tiempo de cierre del contrato en años
    Option_Type: Tipo de opción a valuar
    Trust_level: Confianza definida para el intervalo de precios
    """
    # Definimos tiempo de ejecución
    start = time.time()
    
    # Resolvemos la ecuación de black scholes para obtener los precios
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    # Definimos la longitud h
    h = T / NbStep
    # Definimos el dataframe de strikes
    strike = K
    
    if Option_Type in ['Call', 'call']:
        # Definimos el trapecio para el call
        Average_trapeze_c = (prices * (2 + (r*h) + ((np.random.randn(NbStep, NbTraj)) * sigma))).expanding().sum()
        
        # Calculamos el call de la opción según la formula obtenida para el trapecio
        call = pd.DataFrame({'Prima': np.exp(-r*T) \
                     *np.fmax((h/2*T) * Average_trapeze_c - strike, 0).mean(axis=1)}, index=t)
        # Intervalo de confianza
        sigma_est_call = call.sem().Prima
        mean_est_call = call.iloc[-1].Prima
        i1_call = st.norm.interval(Trust_level, loc=mean_est_call, scale=sigma_est_call)
        
        end_c = time.time()
        total_time_c = end_c - start

        return call.iloc[-1].Prima, i1_call[0], i1_call[1], i1_call[1] - i1_call[0], total_time_c
    
    else:
        # Definimos el trapecio para el put
        Average_trapeze_p = (prices * (2 + (r*h) + ((np.random.randn(NbStep, NbTraj)) * sigma))).expanding().sum()
        
        # Calculamos el put de la opción según la formula obtenida para el trapecio
        put = pd.DataFrame({'Prima': np.exp(-r*T) \
                     *np.fmax(strike - (h/2*T) * Average_trapeze_p, 0).mean(axis=1)}, index=t)
        # Intervalo de confianza
        sigma_est_put = put.sem().Prima
        mean_est_put = put.iloc[-1].Prima
        i1_put = st.norm.interval(Trust_level, loc=mean_est_put, scale=sigma_est_put)
        
        end_p = time.time()
        total_time_p = end_p - start
    
        return put.iloc[-1].Prima, i1_put[0], i1_put[1], i1_put[1] - i1_put[0], total_time_p

$$\hat V_0^{(1)}= {e^{-rT} \over M} \sum_{j=1}^{M} \Bigg({1\over N} \sum_{i=0}^{N-1} S_{t_i}-K \Bigg)_+$$

In [15]:
# Función Riemann.
def Riemann_approach(K:'Strike price',
                     r:'Tasa libre de riesgo',
                     S0:'Precio inicial',
                     NbTraj:'Número trayectorias',
                     NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad',T:'Tiempo de cierre del contrato en años',
                     Option_Type: 'Tipo de opción a valuar. Call o Put.',
                     Trust_level: 'Confianza definida para el intervalo de precios',
                     flag=None):
    # Definimos tiempo de ejecución
    start = time.time()
    
    # Resolvemos la ecuación de black scholes para obtener los precios
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    # Obtenemos los precios promedios
    Average_t = prices.expanding().mean()
    # Definimos el dataframe de strikes
    strike = K
    
    if Option_Type in ['Call', 'call']:
        # Calculamos el call de la opción según la formula obtenida para Sumas de Riemann
        call = pd.DataFrame({'Prima': np.exp(-r*T) \
                     *np.fmax(Average_t - strike, 0).mean(axis=1)}, index=t)
        # Intervalo de confianza
        sigma_est_call = call.sem().Prima
        mean_est_call = call.iloc[-1].Prima
        i1_call = st.norm.interval(Trust_level, loc=mean_est_call, scale=sigma_est_call)
        
        end_c = time.time()
        total_time_c = end_c - start

        return call.iloc[-1].Prima, i1_call[0], i1_call[1], i1_call[1] - i1_call[0], total_time_c
    
    else:
        # Calculamos el put de la opción según la formula obtenida para Sumas de Riemann
        put = pd.DataFrame({'Prima': np.exp(-r*T) \
                     *np.fmax(strike - Average_t, 0).mean(axis=1)}, index=t)
        # Intervalo de confianza
        sigma_est_put = put.sem().Prima
        mean_est_put = put.iloc[-1].Prima
        i1_put = st.norm.interval(Trust_level, loc=mean_est_put, scale=sigma_est_put)
        
        end_p = time.time()
        total_time_p = end_p - start

        return put.iloc[-1].Prima, i1_put[0], i1_put[1], i1_put[1] - i1_put[0], total_time_p

In [16]:
# Parámetros
N_traj = [1000]*3+[5000]*3+[10000]*3+[50000]*3+[100000]*3+[500000]*3+[1000000]*3
N_step = [10,50,100]*len(set(N_traj))
index = [N_traj, N_step]

N_traj_sim = [1000, 5000, 10000, 50000, 100000, 500000, 1000000]
N_step_sim = [10, 50, 100]

# Valores de la opción.
S0 = 100
K = 100
r = 0.10
sigma = 0.20
T = 1

In [17]:
# Función para concatenar las trayectorias.
def valores_matriz(simulacion):
    
    # Trayectorias
    t1 = ([np.array(simulacion[i]) for i in range(len(simulacion))][0])
    t2 = ([np.array(simulacion[i]) for i in range(len(simulacion))][1])
    t3 = ([np.array(simulacion[i]) for i in range(len(simulacion))][2])
    t4 = ([np.array(simulacion[i]) for i in range(len(simulacion))][3])
    t5 = ([np.array(simulacion[i]) for i in range(len(simulacion))][4])
    t6 = ([np.array(simulacion[i]) for i in range(len(simulacion))][5])
    t7 = ([np.array(simulacion[i]) for i in range(len(simulacion))][6])
    m = np.concatenate([t1,t2,t3,t4,t5,t6,t7])
    
    return m

### Método Trapecio: `call asiática`

In [18]:
# Simulación para call con Método del Trapecio. 
trapecio_call_sim = [[Trapecio(K,r,S0,j,i,sigma,T,'Call',0.95) for i in N_step_sim] for j in N_traj_sim]                                         

# Data frame con la información.
trapecio_call = pd.DataFrame(valores_matriz(trapecio_call_sim),
                                 columns=['Aproximación', 'Linferior', 'Lsuperior', 'Longitud al 95%', 'Tiempo (mm:ss)'],
                                 index=pd.MultiIndex.from_arrays(index, names=('Tray. Montecarlo', 'Num. pasos en el tiempo')))
trapecio_call

Aproximación  Linferior  \
Tray. Montecarlo Num. pasos en el tiempo                            
1000             10                              7.332      5.892   
                 50                              7.298      6.864   
                 100                             7.120      6.843   
5000             10                              6.930      5.571   
                 50                              6.922      6.515   
                 100                             7.113      6.833   
10000            10                              6.727      5.409   
                 50                              6.935      6.527   
                 100                             7.157      6.874   
50000            10                              6.885      5.535   
                 50                              6.984      6.571   
                 100                             7.008      6.733   
100000           10                              6.870      5.523   
                 50                              6.973      6.561   
                 100                             7.033      6.757   
500000           10                              6.898      5.545   
                 50                              7.015      6.601   
                 100                             7.036      6.759   
1000000          10                              6.909      5.554   
                 50                              7.014      6.600   
                 100                             7.027      6.750   

                                          Lsuperior  Longitud al 95%  \
Tray. Montecarlo Num. pasos en el tiempo                               
1000             10                           8.772            2.879   
                 50                           7.732            0.868   
                 100                          7.397            0.555   
5000             10                           8.289            2.718   
                 50                           7.330            0.814   
                 100                          7.392            0.559   
10000            10                           8.045            2.637   
                 50                           7.344            0.817   
                 100                          7.440            0.566   
50000            10                           8.236            2.701   
                 50                           7.397            0.825   
                 100                          7.284            0.551   
100000           10                           8.218            2.695   
                 50                           7.384            0.822   
                 100                          7.310            0.553   
500000           10                           8.251            2.706   
                 50                           7.430            0.829   
                 100                          7.312            0.553   
1000000          10                           8.264            2.710   
                 50                           7.428            0.828   
                 100                          7.303            0.553   

                                          Tiempo (mm:ss)  
Tray. Montecarlo Num. pasos en el tiempo                  
1000             10                                0.089  
                 50                                0.034  
                 100                               0.037  
5000             10                                0.104  
                 50                                0.132  
                 100                               0.169  
10000            10                                0.203  
                 50                                0.261  
                 100                               0.339  
50000            10                                1.217  
                 50                                1.263  
                 100          

### Riemann: `call asiática`

In [9]:
# Simulación para call con Riemann.
riemann_call_sim = [[Riemann_approach(K,r,S0,j,i,sigma,T,'Call', 0.95) for i in N_step_sim] for j in N_traj_sim]

# Data frame con la información.
riemann_call = pd.DataFrame(valores_matriz(riemann_call_sim),
                                 columns=['Aproximación', 'Linferior', 'Lsuperior', 'Longitud al 95%', 'Tiempo(mm:ss)'],
                                 index=pd.MultiIndex.from_arrays(index, names=('Tray. Montecarlo', 'Num. pasos en el tiempo')))
riemann_call 

Aproximación  Linferior  \
Tray. Montecarlo Num. pasos en el tiempo                            
1000             10                              6.507      5.206   
                 50                              7.272      6.730   
                 100                             6.864      6.515   
5000             10                              6.542      5.243   
                 50                              7.007      6.485   
...                                                ...        ...   
500000           50                              6.908      6.393   
                 100                             6.981      6.622   
1000000          10                              6.412      5.140   
                 50                              6.920      6.403   
                 100                             6.996      6.635   

                                          Lsuperior  Longitud al 95%  \
Tray. Montecarlo Num. pasos en el tiempo                               
1000             10                           7.807            2.601   
                 50                           7.814            1.084   
                 100                          7.212            0.697   
5000             10                           7.841            2.598   
                 50                           7.528            1.043   
...                                             ...              ...   
500000           50                           7.424            1.031   
                 100                          7.341            0.719   
1000000          10                           7.683            2.543   
                 50                           7.436            1.033   
                 100                          7.357            0.722   

                                          Tiempo(mm:ss)  
Tray. Montecarlo Num. pasos en el tiempo                 
1000             10                               0.057  
                 50                               0.062  
                 100                              0.065  
5000             10                               0.271  
                 50                               0.284  
...                                                 ...  
500000           50                              38.982  
                 100                             35.533  
1000000          10                              54.764  
                 50                              55.593  
                 100                             66.026  

[21 rows x 5 columns]

### Método Trapecio: `put asiática`

In [10]:
# Simulación para put con Método del Trapecio.                                          
trapecio_put_sim = [[Trapecio(K,r,S0,j,i,sigma,T,'Put',0.95) for i in N_step_sim] for j in N_traj_sim]

# Data frame con la información.
trapecio_put = pd.DataFrame(valores_matriz(trapecio_put_sim),
                                 columns=['Aproximación', 'Linferior', 'Lsuperior', 'Longitud al 95%', 'Tiempo (mm:ss)'],
                                 index=pd.MultiIndex.from_arrays(index, names=('Tray. Montecarlo', 'Num. pasos en el tiempo')))
trapecio_put

Aproximación  Linferior  \
Tray. Montecarlo Num. pasos en el tiempo                            
1000             10                              2.223    -14.913   
                 50                              2.209     -5.272   
                 100                             2.388     -2.858   
5000             10                              2.195    -14.923   
                 50                              2.375     -5.086   
...                                                ...        ...   
500000           50                              2.334     -5.127   
                 100                             2.340     -2.918   
1000000          10                              2.226    -14.890   
                 50                              2.333     -5.130   
                 100                             2.341     -2.917   

                                          Lsuperior  Longitud al 95%  \
Tray. Montecarlo Num. pasos en el tiempo                               
1000             10                          19.359           34.272   
                 50                           9.690           14.962   
                 100                          7.633           10.491   
5000             10                          19.314           34.237   
                 50                           9.835           14.921   
...                                             ...              ...   
500000           50                           9.796           14.923   
                 100                          7.598           10.516   
1000000          10                          19.342           34.231   
                 50                           9.795           14.925   
                 100                          7.599           10.515   

                                          Tiempo (mm:ss)  
Tray. Montecarlo Num. pasos en el tiempo                  
1000             10                                0.100  
                 50                                0.115  
                 100                               0.074  
5000             10                                0.270  
                 50                                0.312  
...                                                  ...  
500000           50                               29.853  
                 100                              36.790  
1000000          10                               50.251  
                 50                               64.285  
                 100                              80.853  

[21 rows x 5 columns]

### Riemann: `put asiática`

In [11]:
# Simulación para call con Riemann.
riemann_put_sim = [[Riemann_approach(K,r,S0,j,i,sigma,T,'Put',0.95) for i in N_step_sim] for j in N_traj_sim]

# Data frame con la información.
riemann_put = pd.DataFrame(valores_matriz(riemann_put_sim),
                                 columns=['Aproximación', 'Linferior', 'Lsuperior', 'Longitud al 95%', 'Tiempo(mm:ss)'],
                                 index=pd.MultiIndex.from_arrays(index, names=('Tray. Montecarlo', 'Num. pasos en el tiempo')))
riemann_put

Aproximación  Linferior  \
Tray. Montecarlo Num. pasos en el tiempo                            
1000             10                              2.114      1.717   
                 50                              2.287      2.142   
                 100                             2.364      2.261   
5000             10                              2.292      1.850   
                 50                              2.251      2.113   
...                                                ...        ...   
500000           50                              2.330      2.183   
                 100                             2.336      2.236   
1000000          10                              2.222      1.796   
                 50                              2.326      2.178   
                 100                             2.347      2.247   

                                          Lsuperior  Longitud al 95%  \
Tray. Montecarlo Num. pasos en el tiempo                               
1000             10                           2.512            0.796   
                 50                           2.431            0.290   
                 100                          2.466            0.205   
5000             10                           2.734            0.884   
                 50                           2.389            0.276   
...                                             ...              ...   
500000           50                           2.478            0.295   
                 100                          2.435            0.199   
1000000          10                           2.648            0.851   
                 50                           2.473            0.295   
                 100                          2.447            0.200   

                                          Tiempo(mm:ss)  
Tray. Montecarlo Num. pasos en el tiempo                 
1000             10                               0.101  
                 50                               0.085  
                 100                              0.092  
5000             10                               0.319  
                 50                               0.466  
...                                                 ...  
500000           50                              26.672  
                 100                             30.671  
1000000          10                            1256.268  
                 50                              74.770  
                 100                             71.764  

[21 rows x 5 columns]

> Los reusltados obtenidos en las tablas anteriores varían un poco. Sin embargo, creo que se pueda considerar como una diferencia insignificativa.

> Respecto a los tiempos en ambos métodos para los dos tipos de opciones son muy similares, por lo que en términos de optimización, es irrelevante el método que se escoja usar.